# LiricAI

Aqui é mostrado um pequeno projecto em que se usa uma Recurrent Neural Network (segundo https://gilberttanner.com/blog/generating-text-using-a-recurrent-neuralnetwork) para criar a letra de uma música a partir de um determinado conjunto de letras já existente. 

## Os dados

Os ficheiros utilizados encontram-se disponíveis no repositório de Github como ficheiros .txt

## Bibliotecas a usar

Como estou a seguir a ligação acima referida, Keras e Tensorflow.

## Tratamento de dados

In [24]:
from os import listdir
from os.path import isfile, join
import sys
import collections, functools, operator
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback, ModelCheckpoint, ReduceLROnPlateau

Contar os caracteres presentes nas letras:

In [25]:
dataset_path = "./dataset"
files_in_dataset = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]
char_indices_list = []
indices_char_list = []

total_lyrics = ""
for filename in files_in_dataset:
    filepath = dataset_path +"/"+filename
    with open(filepath, 'r',encoding='utf8') as file:

        text = file.read().lower()
      
        total_lyrics += "\n\n"+text 

chars = sorted(list(set(total_lyrics))) # getting all unique chars

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(char_indices)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, '+': 7, ',': 8, '-': 9, '.': 10, '/': 11, '2': 12, '3': 13, '6': 14, '7': 15, '9': 16, ':': 17, ';': 18, '?': 19, '[': 20, ']': 21, 'a': 22, 'b': 23, 'c': 24, 'd': 25, 'e': 26, 'f': 27, 'g': 28, 'h': 29, 'i': 30, 'j': 31, 'k': 32, 'l': 33, 'm': 34, 'n': 35, 'o': 36, 'p': 37, 'q': 38, 'r': 39, 's': 40, 't': 41, 'u': 42, 'v': 43, 'w': 44, 'x': 45, 'y': 46, 'z': 47, '{': 48, '}': 49, 'à': 50, 'á': 51, 'â': 52, 'ã': 53, 'ç': 54, 'é': 55, 'ê': 56, 'í': 57, 'ó': 58, 'ô': 59, 'õ': 60, 'ú': 61, '\u2005': 62, '’': 63, '…': 64, '\u205f': 65}


In [26]:
max_len = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(total_lyrics) - max_len, step):
    sentences.append(total_lyrics[i: i + max_len])
    next_chars.append(total_lyrics[i + max_len])

x = np.zeros((len(sentences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## Recurrent Neural Network

In [27]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_len, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [28]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - max_len - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + max_len]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, max_len, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [29]:
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [30]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

callbacks = [print_callback, checkpoint, reduce_lr]

## Treinar a rede

In [31]:
model.fit(x, y, batch_size=128, epochs=40, callbacks=callbacks)

Epoch 1/40
276/276 [==============================] - ETA: 0s - loss: 2.4178
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ro ao boneco
gigante barrado no beco

eu"
ro ao boneco
gigante barrado no beco

eu sem a sem sem tem a sem a perte a mem sem sem a perta a mem sem te a tem sem sem a carta a mem a cara a camar

e que o sem este a sem que o com a sem a sem a camar

e que o sem sem a sem te a sem a camar

a corta a mem de tem a camar

e que este tem aste a cara cama a cantar

a mem a cara a sem a sente a corta a arar

a mempre a corte a mem a mem te a sem sem a sem a corte a mem a per a sem a car
----- diversity: 0.5
----- Generating with seed: "ro ao boneco
gigante barrado no beco

eu"
ro ao boneco
gigante barrado no beco

eu sem veito te o que faca o porte chem te o tem empera o fiste esteu de arora temo me o te te as sente vim a cor
e sem
esto a te estim

cempara te o para mem malade
sem a tegre a prem a perte a camor e para cora com sem a 

e daqui
eu dosehá de bicar
tm-cress
o qle é tanto sondem mei a sei quere nos querer
que nos balnas om tupeira à efravar
vou de lenhe de apalo
teu amar no pispebe
naquerno o joguça
não tunho o ou, se quei irro se tenho poe por na pro que es moupor
segue o conlinho à voi
lá taris

quem se te a

Epoch 00007: loss improved from 1.39590 to 1.30392, saving model to weights.hdf5
276/276 [==============================] - 62s 226ms/step - loss: 1.3039
Epoch 8/40
275/276 [============================>.] - ETA: 0s - loss: 1.2309
----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: "tiro ao boneco
gigante barrado no beco

"
tiro ao boneco
gigante barrado no beco

eu sei se me mundo for de porte
se ou esqueco ser um pressa a canção
trocar a alia e espera a terra
se te hoja amor tem premer
não te caia a manhã
não se eu sei se num a sater
a morte no teu ar o que eu teu santo para o teu senter

eu tou vem a caração
não se eu sei que não vai de mim?
e o mundo com um pr

que o noberes que a gortar

mas tu estás aqui de niação 
tão tenverámos de sarrar

sento as redento p'ra alivante canção
não com espinca do pouto
veltanjueu locar e que devia ivê que nos sabes ela ós)

escerámo ao dao campanç
----- diversity: 1.2
----- Generating with seed: "m baixo ficava marreco
estou tão em-mim-"
m baixo ficava marreco
estou tão em-mim-m
vem tmos das comesçandes elchada
more, ao despozem
segures que nós
açora, ar)rmos que tenteá no tero

não conselio a pepa, fazes mas dessenair a zulhe
este ceisa que sheva ser fofis iofim das dentro
forças a morte imuão todiande lovarga
nastida à bra veztes
avia lá mario até que serer
tado, mais à batadar
está mais ai do trece para véu irstinl
um jabar aqui
só fio e a deva a riza
hoi que tenho u

Epoch 00014: loss improved from 1.01298 to 0.98682, saving model to weights.hdf5
276/276 [==============================] - 58s 209ms/step - loss: 0.9868
Epoch 15/40
276/276 [==============================] - ETA: 0s - loss: 0.9606
----- Ge

mudo que é tenho ao chão
que não sou semva eu som
eu nome descado no teus olhos do campo
pra rim
dar o que serer
como suidade no bom ser fugar
os deste o tempo fomem mais uma erado
quando te forja esco
----- diversity: 1.0
----- Generating with seed: "tante
tudo livros de engordar
e eu preci"
tante
tudo livros de engordar
e eu precisto a voz

eu deu peisa a casar como fosteu
quando o estreiros que tens como tanto faz
rtindo falea de me dar
dar, dar
não é bem hamistida quel de quiser
mos ela venta camnos sempre melênjura
mas tão sem como a vida

espindos nem ser legar
em cresa passa para paras lonter
todo a lado vem dizer
tenha memer da ti
da lade-eito
que não, não, não vais pequer'ar
mão me muda de cim-teça
todo o meu almo t
----- diversity: 1.2
----- Generating with seed: "tante
tudo livros de engordar
e eu preci"
tante
tudo livros de engordar
e eu precisto jembre a suarda que fim

resqueler-moue na valhavia de contar

quases fagos vaio foco
inte permi
mederessa o barco do quer sorrir

vou palos não se muda à tempo de muda
na ladei a tratina

que esta tenho que está de mim
por tu nas, ligade e no meu cansa para o mar
se do meu cant
----- diversity: 0.5
----- Generating with seed: "a
tenho os braços no estendal
eu podia a"
a
tenho os braços no estendal
eu podia achar
na minha passa perder
podes vira, ageessa não vai de quero osti
os mora o tempo portis
de outro livarto

a pretes enhado a teu conhe estás almo te conseguria
que te caia encunda e tu faz
todo o sento e o sol 

e que não sabo que o só
com ser se tu não se auto ao proflhão

menina de lada é o sondo com essa a caua para o maria

não se esperar alguém a casim no lado

eu sem estou a teu precebe d
----- diversity: 1.0
----- Generating with seed: "a
tenho os braços no estendal
eu podia a"
a
tenho os braços no estendal
eu podia acorada
maninao falar, como tudo o que tinha maio ser 
trazes atrás aquela
o mais de ricabelendo

vai o sento o mar
estar a id'iba
é essa assim se sabe por de outro de sentiregrerr
quemeé

275/276 [============================>.] - ETA: 0s - loss: 0.6901
----- Generating text after Epoch: 34
----- diversity: 0.2
----- Generating with seed: "e
no pico não peco
lá em baixo ficava ma"
e
no pico não peco
lá em baixo ficava marinho

e que estou a morte, acarte
e a vida nos diar
se hou a ladei a sui
prensssolondo é só só sei crescer

e assim não sabe ançarmela, me queres mais de mim do chão

e quando a terra e tem é demais
ais como um hora a vida que é querer à imbraço assim
mais lidásia
a calças, não sabe a dizão

a chupram que não vento
que nos mudo que esta large de alguém
gasso amanhe no cheiro que não vam
quero se 
----- diversity: 0.5
----- Generating with seed: "e
no pico não peco
lá em baixo ficava ma"
e
no pico não peco
lá em baixo ficava maris
larco eu diande noite em ti
e tu aqui
estava eu a pensar agora em ti
e tu aqui
e tu aquile consegui arceu
com uma passoar a estrecer
dar e raivar de p'ra lada a chuém

a voz vencado sombes a marinheir
ainda que me cansendia
se 

In [32]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - max_len - 1)
    generated = ''
    sentence = text[start_index: start_index + max_len]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, max_len, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [56]:
import datetime
number_of_files = 10

date = datetime.datetime.now().date()

for counter in range(0, number_of_files):
    songFileName = "results/output_"+str(date)+"_"+str(counter)+".txt"
    with open(songFileName, 'w') as f:
        f.write(generate_text(500, 0.2))

#print(generate_text(500, 0.2))